## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-03-20-50-00 +0000,wsj,Opinion | Making a Palestinian State Less Likely,https://www.wsj.com/opinion/palestinian-state-...
1,2025-08-03-20-49-45 +0000,nyt,Texas Democrats to Leave State to Stop Republi...,https://www.nytimes.com/2025/08/03/us/texas-de...
2,2025-08-03-20-48-41 +0000,nypost,Single mom knocked out cold in Cincinnati mob ...,https://nypost.com/2025/08/03/us-news/single-m...
3,2025-08-03-20-47-00 +0000,wsj,Opinion | Ukrainians’ Anticorruption Victory,https://www.wsj.com/opinion/ukraine-anticorrup...
4,2025-08-03-20-46-40 +0000,nypost,Amy Bradley case hinges on ‘somebody rememberi...,https://nypost.com/2025/08/03/us-news/amy-brad...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
76,trump,33
31,new,10
17,attack,8
360,russian,7
329,oil,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
94,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...,82
167,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...,76
156,2025-08-03-00-48-00 +0000,wsj,Erika McEntarfer led the Bureau of Labor Stati...,https://www.wsj.com/politics/policy/the-econom...,62
149,2025-08-03-02-30-01 +0000,startribune,Senate heads home with no deal to speed confir...,https://www.startribune.com/senate-heads-home-...,61
148,2025-08-03-02-53-15 +0000,nypost,Trump tells Chuck Schumer to ‘GO TO HELL’ over...,https://nypost.com/2025/08/02/us-news/trump-te...,58


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
94,82,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...
35,45,2025-08-03-18-39-45 +0000,nypost,Montana bar shooting victims named — as top co...,https://nypost.com/2025/08/03/us-news/montana-...
110,45,2025-08-03-10-34-36 +0000,nypost,Ukrainian drone attack sparks massive fire at ...,https://nypost.com/2025/08/03/world-news/ukrai...
167,43,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...
23,32,2025-08-03-19-44-44 +0000,nypost,Accused NYC crypto bro kidnappers bragged abou...,https://nypost.com/2025/08/03/us-news/accused-...
149,28,2025-08-03-02-30-01 +0000,startribune,Senate heads home with no deal to speed confir...,https://www.startribune.com/senate-heads-home-...
150,27,2025-08-03-02-00-00 +0000,wsj,The build-out of AI infrastructure is costing ...,https://www.wsj.com/economy/the-ai-booms-hidde...
143,26,2025-08-03-04-27-43 +0000,bbc,"Their children can't eat, speak or walk - so f...",https://www.bbc.com/news/articles/c39d0zj0110o...
24,25,2025-08-03-19-40-51 +0000,nyt,Third Child in Sailing Camp Dies After Barge H...,https://www.nytimes.com/2025/08/03/us/miami-be...
78,24,2025-08-03-15-27-50 +0000,bbc,Aid group says worker killed by Israeli milita...,https://www.bbc.com/news/articles/cx2x5eyl676o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
